1

In [ ]:
import re
text = 'kjs jana 182'
re.sub(r'1{0+}|([1-9]\d{3,})', '', text)

'kjs jana 182'

2

In [ ]:
from gensim.test.utils import common_texts
from gensim.models import Word2Vec

model = Word2Vec(sentences=common_texts, vector_size=100, window=5, min_count=1, workers=4)
model.save("word2vec.model")


In [ ]:
loaded_model = Word2Vec.load("word2vec.model")
loaded_model.wv['computer']

array([-0.00515774, -0.00667028, -0.0077791 ,  0.00831315, -0.00198292,
       -0.00685696, -0.0041556 ,  0.00514562, -0.00286997, -0.00375075,
        0.0016219 , -0.0027771 , -0.00158482,  0.0010748 , -0.00297881,
        0.00852176,  0.00391207, -0.00996176,  0.00626142, -0.00675622,
        0.00076966,  0.00440552, -0.00510486, -0.00211128,  0.00809783,
       -0.00424503, -0.00763848,  0.00926061, -0.00215612, -0.00472081,
        0.00857329,  0.00428459,  0.0043261 ,  0.00928722, -0.00845554,
        0.00525685,  0.00203994,  0.0041895 ,  0.00169839,  0.00446543,
        0.0044876 ,  0.0061063 , -0.00320303, -0.00457706, -0.00042664,
        0.00253447, -0.00326412,  0.00605948,  0.00415534,  0.00776685,
        0.00257002,  0.00811905, -0.00138761,  0.00808028,  0.0037181 ,
       -0.00804967, -0.00393476, -0.0024726 ,  0.00489447, -0.00087241,
       -0.00283173,  0.00783599,  0.00932561, -0.0016154 , -0.00516075,
       -0.00470313, -0.00484746, -0.00960562,  0.00137242, -0.00

3

In [ ]:
import torch.nn as nn
import torch

class LSTMClassification(nn.Module):
  def __init__(self, input_size, output_size, hidden_size, n_layers):
    self.n_layers = n_layers
    self.lstm = nn.LSTM(input_size, hidden_size)
    self.linear = nn.Linear(hidden_size, output_size)

  def forward(self, x_batch):
    out, (hidden_state, cell_state) = self.lstm(x_batch)
    out = self.linear(out)

4

In [ ]:
class Encoder:
  def __init__(self, inp_size, hid_size):
    self.emb = nn.Embedding(inp_size, hid_size)
    self.gru = nn.GRU(inp_size, hid_size, batch_first=True)

  def forward(self, x_batch):
    emb = self.emb(x_batch)
    out, hidden = self.gru(emb)
    return out, hidden


5

In [ ]:


class Decoder:
  def __init__(self, hid_size, out_size):
    self.emb = nn.Embedding(out_size, hid_size)
    self.gru = nn.GRU(hid_size, hid_size, batch_first=True)
    self.out = nn.Linear(hid_size, out_size)

  def forward(self, encoder_out, encoder_hid):
    batch_size = encoder_out.size(0)
    decoder_input = torch.empty(batch_size, 1, dtype=torch.long, device=device).fill(SOS)

    decoder_hid = encoder_hid

    decoder_outs = []

    for i in range(MAX_LEN):
      decoder_out, decoder_hidden = self.forward_step(decoder_input, decoder_hid)
      decoder_outs.append(decoder_out)

      _, topi = decoder_out.topk(1)
      decoder_input = topi.squeeze(-1).detach()

    decoder_outs = torch.cat(decoder_outs, dim=1)
    decoder_outs = F.log_softmax(decoder_outs, dim=1)
    return decoder_outs, decoder_hid, None

  def forward_step(self, inp, hid):
    out = self.emb(inp)
    out = F.relu(out)
    out, hid = self.gru(out, hid)
    out = self.out(out)
    return out, hid


6

In [ ]:
from torchdata import datapipes
from nltk.tokenize import word_tokenize
from torchtext.vocab import build_vocab_from_iterator


data_pipe = datapipes.iter.IterableWrapper(['name'])
data_pipe = datapipes.iter.FileOpener(data_pipe, mode='r', encoding='utf-8')
data_pipe = data_pipe.readlines()

def tokenize_sentence(text):
  return word_tokenize(text, language="russian")

def tokenize_data_pipe(data_pipe, tokenizer):
  tokenized_data = []
  for data in data_pipe:
    tokenized_data.append(tokenizer(data[1]))
  return tokenized_data

def yield_tokens(data_iter: datapipes.iter.IterDataPipe, tokenizer):
  for sent in data_iter:
    yield tokenizer(sent[1])

vocab = build_vocab_from_iterator(
    yield_tokens(data_pipe, tokenizer=tokenize_sentence),
    min_freq=1,
    specials= ['<pad>', '<sos>', '<eos>', '<unk>'],
    special_first=True
)
vocab.set_default_index(vocab['<unk>'])

7

In [ ]:
from transformers import pipeline

pipe = pipeline("text-generation", model="tinkoff-ai/ruDialoGPT-small")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/755 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/551M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.71M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/107 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/379 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
pipe('@@ПЕРВЫЙ@@ привет @@ВТОРОЙ@@ привет @@ПЕРВЫЙ@@ как дела? @@ВТОРОЙ@@')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


[{'generated_text': '@@ПЕРВЫЙ@@ привет @@ВТОРОЙ@@ привет @@ПЕРВЫЙ@@ как дела? @@ВТОРОЙ@@Привет, как дела?  Как дела'}]

8

In [ ]:
class SelfAttention:
  def __init__(self, emb_dim):
    self.selfattn = nn.MultiheadAttention(emb_dim, 1)
    self.layer_norm = nn.LayerNorm(emb_dim)

  def forward(self, query, key, value):
    attn_output, attn_weights = self.selfattn(query, key, value)
    out = self.layer_norm(attn_output + query)

    return out, attn_weights

9

In [ ]:
! pip install bigartm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.0 MB/s eta 0:00:00


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from artm import BatchVectorizer


In [ ]:
from datasets import load_dataset
import nltk
import string
from nltk.corpus import stopwords
from sklearn.datasets import fetch_20newsgroups
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
import pandas as pd
import artm

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

dataset = fetch_20newsgroups(remove=('headers', 'footers', 'qutes', 'target'))
df = pd.DataFrame({'text': dataset.data})


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
df

,text
0,I was wondering if anyone out there could enli...
1,A fair number of brave souls who upgraded thei...
2,"well folks, my mac plus finally gave up the gh..."
3,Robert J.C. Kyanko (rob@rjck.UUCP) wrote:\n> a...
4,"From article <C5owCB.n3p@world.std.com>, by to..."
...,...
11309,DN> From: nyeda@cnsvax.uwec.edu (David Nye)\nD...
11310,"I have a (very old) Mac 512k and a Mac Plus, b..."
11311,I just installed a DX2-66 CPU in a clone mothe...
11312,In article <1qkgbuINNs9n@shelley.u.washington....


In [ ]:
stopwords_list = set(stopwords.words('english'))

def preprocess(text: str) -> str:
    text = text.lower()

    word_list = word_tokenize(text)
    stopwords_list = set(stopwords.words('english'))

    word_list = [word for word in word_list if word not in stopwords_list]
    word_list = [word for word in word_list if len(word) > 2]
    word_list = [word for word in word_list if word not in string.punctuation]

    lemma = WordNetLemmatizer()
    word_list = [lemma.lemmatize(word) for word in word_list]
    text= ' '.join(word_list)

    return text

df['text'] = df['text'].apply(lambda x: preprocess(x))

In [ ]:
vectorizer = CountVectorizer(ngram_range=(1,3))
m_train = vectorizer.fit_transform(df.head(1000)['text'])

vocabulary = vectorizer.get_feature_names_out()
n_wd = m_train.todense().T
batch_vectorizer = BatchVectorizer(data_format='bow_n_wd', n_wd=n_wd, vocabulary=vocabulary, target_folder='batches')

lda = artm.LDA(num_topics=20, alpha=0.1, beta=0.1, num_document_passes=5, dictionary=batch_vectorizer.dictionary)
lda.fit_offline(batch_vectorizer, num_collection_passes=20)
lda.perplexity_last_value

21464.634765625

10

In [ ]:
! pip install bertopic -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.1/154.1 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 23.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.9/90.9 kB 12.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 11.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 7.0 MB/s eta 0:00:00


In [ ]:
from bertopic import BERTopic

In [ ]:
df_list = df.head(1000)['text'].apply(lambda x: str(x))

In [ ]:
topic_model = BERTopic(embedding_model='distiluse-base-multilingual-cased-v1', n_gram_range=(1,3), top_n_words=6, calculate_probabilities=True)
topics, probs = topic_model.fit_transform(df_list)
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,142,-1_ax_ax ax_ax ax ax_max,"[ax, ax ax, ax ax ax, max, max ax, ax max]",[final public update dragon magazine auction n...
1,0,281,0_window_problem_file_get,"[window, problem, file, get, edu, thanks]",[dxb132 psuvm.psu.edu writes article 1qlbrlinn...
2,1,96,1_game_team_play_year,"[game, team, play, year, player, season]",[opi offensive production index player least a...
3,2,92,2_jesus_god_one_christian,"[jesus, god, one, christian, people, argument]",[bissda saturn.wwc.edu dan lawrence bissell wr...
4,3,87,3_car_com_bike_article,"[car, com, bike, article, writes, would]",[article 1qkon8 3re armory.centerline.com jimf...
5,4,52,4_health_tobacco_use_smokeless,"[health, tobacco, use, smokeless, smokeless to...",[article noringc5snsx.kmo netcom.com noring ne...
6,5,48,5_com_article_writes_edu,"[com, article, writes, edu, writes article, mo...",[article c5x3l0.3r8 athena.cs.uga.edu mcovingt...
7,6,41,6_key_chip_phone_clipper,"[key, chip, phone, clipper, encryption, would]",[article 1993apr19.182327.3420 guvax.acc.georg...
8,7,33,7_gun_weapon_would_people,"[gun, weapon, would, people, state, law]",[iftccu talk.politics.guns vincent cad.gatech....
9,8,29,8_space_nasa_shuttle_mission,"[space, nasa, shuttle, mission, astronaut, orbit]",[archive-name space/schedule last-modified dat...
